In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,44845
2,Huelva,Confirmados PDIA 14 días,606
3,Huelva,Tasa PDIA 14 días,"118,08952199076329"
4,Huelva,Confirmados PDIA 7 días,222
5,Huelva,Tasa PDIA 7 dias,"43,2605179570123"
6,Huelva,Total Confirmados,45044
7,Huelva,Curados,41323
8,Huelva,Fallecidos,410


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  44845.0


/tmp/ipykernel_3667/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12412.0


/tmp/ipykernel_3667/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_3667/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_3667/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_3667/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 4230 personas en los últimos 7 días 

Un positivo PCR cada 1169 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,44845.0,222.0,606.0,513170.0,43.260518,118.089522,48.0
Huelva-Costa,26536.0,166.0,408.0,289548.0,57.330736,140.909279,42.0
Cartaya,1924.0,49.0,79.0,20083.0,243.987452,393.367525,18.0
Ayamonte,2022.0,22.0,45.0,21104.0,104.245641,213.229719,7.0
Isla Cristina,3089.0,32.0,62.0,21393.0,149.581639,289.814425,7.0
Lepe,2975.0,14.0,38.0,27880.0,50.215208,136.298422,4.0
Condado-Campiña,13922.0,31.0,138.0,156231.0,19.842413,88.330741,3.0
Huelva (capital),12412.0,34.0,123.0,143837.0,23.637868,85.513463,3.0
Punta Umbría,1255.0,4.0,16.0,15355.0,26.050147,104.200586,2.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Cañaveral de León,32.0,1.0,3.0,396.0,252.525253,757.575758,0.0
Higuera de la Sierra,64.0,2.0,6.0,1291.0,154.918668,464.756003,0.0
Villablanca,284.0,5.0,13.0,2885.0,173.310225,450.606586,0.0
Cartaya,1924.0,49.0,79.0,20083.0,243.987452,393.367525,18.0
Aroche,127.0,8.0,12.0,3054.0,261.951539,392.927308,1.0
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0
Isla Cristina,3089.0,32.0,62.0,21393.0,149.581639,289.814425,7.0
Ayamonte,2022.0,22.0,45.0,21104.0,104.245641,213.229719,7.0
Huelva-Costa,26536.0,166.0,408.0,289548.0,57.330736,140.909279,42.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Gibraleón,923.0,1.0,12.0,12737.0,7.851142,94.213708,0.0,0.083333
Puebla de Guzmán,174.0,1.0,9.0,3092.0,32.341527,291.073739,0.0,0.111111
Moguer,1928.0,4.0,20.0,21867.0,18.292404,91.462020,0.0,0.200000
Aljaraque,1610.0,2.0,9.0,21474.0,9.313589,41.911148,1.0,0.222222
Palma del Condado (La),1414.0,2.0,9.0,10801.0,18.516804,83.325618,0.0,0.222222
Condado-Campiña,13922.0,31.0,138.0,156231.0,19.842413,88.330741,3.0,0.224638
Punta Umbría,1255.0,4.0,16.0,15355.0,26.050147,104.200586,2.0,0.250000
Bonares,454.0,1.0,4.0,6060.0,16.501650,66.006601,0.0,0.250000
Huelva (capital),12412.0,34.0,123.0,143837.0,23.637868,85.513463,3.0,0.276423
